In [30]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import numpy as np
import pandas as pd
import yfinance as yf
import filecmp
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)
from sklearn import preprocessing as pre
import sqlite3
from sqlalchemy import create_engine

In [31]:
data=pd.DataFrame()

conection=sqlite3.connect('database.db')
db_path = "database.db"

# Create the engine connection
engine = create_engine(f"sqlite:///{db_path}")

# Specify the table name
table_name = "stocks"

# Read the data into a DataFrame
data = pd.read_sql(f"SELECT * FROM {table_name}", engine)
data


,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,AAPL_Adj Close,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,...,XAUUSD.PEPPERSTONE_Low,XAUUSD.PEPPERSTONE_Close,XAUUSD.PEPPERSTONE_Volume,XAUUSD.PEPPERSTONE_Adj Close,LCO.COMM_Open,LCO.COMM_High,LCO.COMM_Low,LCO.COMM_Close,LCO.COMM_Volume,LCO.COMM_Adj Close
0,2.5850,2.6696,2.5804,2.2722,807234400.0,2.2722,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.6832,2.7136,2.6607,2.2789,619603600.0,2.2789,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.6725,2.6750,2.6339,2.2609,449422400.0,2.2609,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.6875,2.7393,2.6625,2.3193,704457600.0,2.3193,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.7404,2.7571,2.7050,2.3118,675040800.0,2.3118,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.35,80.37,78.88,79.07,119189.0,79.07
18735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.04,81.72,78.76,81.07,59735.0,81.07
18736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,80.79,81.33,79.29,79.65,70385.0,79.65
18737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.80,79.95,78.25,78.39,18782.0,78.39


In [32]:

data_corection=torch.as_tensor(data.values,dtype=torch.float32)
data_corection=data_corection.to(device)
data_corection=F.normalize(data_corection,p=2.0)

data_row=data_corection.t()
data_row=data_row.to(device)
data_corection=(data_corection-torch.min(data_corection))/(torch.max(data_corection)-torch.min(data_corection))
data_corection

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])

In [33]:
class AI(nn.Module):
    def __init__(self,nUnits,nLayers,input_size):
        super(AI,self).__init__()
        
        # create dictionary to store the layers
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers

        ### input layer
        self.layers['input'] = nn.Linear(input_size[0],input_size[1])
        self.layers['hidden0']=nn.Linear(input_size[1],nUnits)
        
        ## hidden layers
        for i in range(nLayers):
            self.layers[f'hidden{i+1}'] = nn.Linear(nUnits,nUnits)
        ### output layer
        self.layers['output'] = nn.Linear(nUnits,1)
    def forward(self,x):
        x=F.relu(self.layers['input'](x))
        for i in range(self.nLayers):
            x=F.relu(self.layers[f'hidden{i}'](x))
        x=self.layers['output'](x)
        return x
        


In [34]:
nUnitsPerLayer=512
nLayers=100
net=AI(1509,nLayers,data_corection.size())
net=net.to(device)


In [35]:
target=data['AAPL_High']
target=torch.tensor(target.values,dtype=torch.float32).to(device)

data_corection.size()

torch.Size([18739, 30])

In [36]:
def trainTheModel(theModel,numepochs):
    lossfun=nn.L1Loss().to(device)
    optimizer=torch.optim.RMSprop(theModel.parameters(),lr=.001,momentum=.9)
    losses=torch.zeros(numepochs).to(device)
    for epochi in range(numepochs):
        print(epochi)
        yHat=theModel(data_corection)
        loss=lossfun(yHat,target)
        losses[epochi]=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(losses)
    return losses

In [37]:
numepochs=1000
losses=trainTheModel(net,numepochs)
losses=losses.to(device='cpu')
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (18739x30 and 18739x30)